#An alternative to Grid Search

Grid Search is a brute force algorithm that helps tunning hyperparameters. Since it is brute force, Grid Search is expensive. To be precise if we want to find the best model out of 1000 different models, using a 5-fold cross validation we need to train 5000 models. 

We present an alternative that uses hypothesis test (binomial test) that only requires to train 1000 models. Thus, we decrease computational costs by 80%. In general if we use $k$-fold cross validation, we decrease computational costs by $\frac{k-1}{k}$%

#The implementation

* We use ```binomtest``` from **scipy** to perform the hypothesis test.
* We train 256 instances of ```GradientBoostingClassifier``` from **sklearn**
* The dataset consists of samples of [Hotel reservations](https://www.kaggle.com/datasets/ahsan81/hotel-reservations-classification-dataset)
* We split the data into train, validation and test sets. The test set will be unseen by our models. We evalute our final model on it.

* We define several functions:

1. ```fit(params,features)``` Fit our (256) models tracking parameters, input features and all metrics we will use

2. ``` bt(models,dataset)``` The binomial test applied to the predictions our models return on a given dataset 

3. ```metric(models)``` A measure of overfitting of models based on binomial test

4. ```drop(models)``` A function to do feature selection over a family of models

5. ```scores(model,features,name)``` Returns a Series containing the scores reached by the model.

* The selected model will be the one that minimizes ```metric(models)```
* We record the time each cell needs to run

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split,GridSearchCV,ParameterGrid


from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score,precision_score,recall_score,f1_score,matthews_corrcoef,make_scorer
from sklearn import metrics

from scipy.stats import binomtest

!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 3.21 ms (started: 2023-03-06 15:19:26 +00:00)


In [26]:
#@title Functions
def fit(params,features):
  fit_models = []
  
  try:
    param_grid = ParameterGrid(params)
    len(np.array(features).shape)==1
    for param in param_grid:
      
      X = X_train1[features]
      VAL = X_val[features]

      gb = GradientBoostingClassifier(**param,random_state=0)
      gb = gb.fit(X,y_train1)
    
      pred_val = gb.predict(VAL) 
      pred_train = gb.predict(X)
    
      fit_models.append({'pred_val':pred_val,'pred_train':pred_train,'param':param,'gb':gb,'features':features})
  except:
    for param, feature in zip(params,features):

      X = X_train1[feature]
      VAL = X_val[feature]

      gb = GradientBoostingClassifier(**param,random_state=0)
      gb = gb.fit(X,y_train1)
    
      pred_val = gb.predict(VAL) 
      pred_train = gb.predict(X)
    
      fit_models.append({'pred_val':pred_val,'pred_train':pred_train,'param':param,'gb':gb,'features':feature})
    
  return fit_models 


def bt(models,dataset):
  for j in models:
    if dataset=='train':
      pred =  j['pred_train']
      trials = len(y_train1)
      p = p_train
    elif dataset=='validation':
      pred =  j['pred_val']
      trials = len(y_val)
      p = p_val
      
    success = len(pred[pred==1])
    model_bt = binomtest(success,trials,p=p,alternative='two-sided')
  
    j['bt_{}'.format(dataset)]=model_bt

def metric(models):
  for m in models:
    per_errtrain = abs(m['bt_train'].proportion_estimate-p_train)
    per_errval = abs(m['bt_validation'].proportion_estimate-p_val)

    m['overfit']=abs(per_errtrain-per_errval)/per_errtrain


def ft_select(models):
  for l in models:
    original = list(zip(df.drop('booking_status',axis=1).columns,l['gb'].feature_importances_))
    original.sort(reverse=True,key=lambda x:x[1])
    l['important_features'] = [y[0] for y in original[:-1]]


def ft_pm_extract(models):
  original_params = [x['param'] for x in models]
  important_features = [x['important_features'] for x in models]
  return original_params, important_features

def refine(models,prev_models):
  new_metrics= [x['overfit'] for x in models]
  old_metrics = [x['overfit'] for x in prev_models]
  to_compare = range(len(models))
  errors = [abs(new_metrics[y]-old_metrics[y]) for y in to_compare]
  
  movement = [new_metrics[y]-old_metrics[y] for y in to_compare]
  

  for m in to_compare:
    models[m]['error'] = movement[m]
    models[m]['absolute_error'] = errors[m]

  #models.sort(key=lambda y:y['error'])

def drop(models):
  ft_select(models)
  old_param, new_feat = ft_pm_extract(models)
  if len(new_feat[0])>0:
    new_models = fit(old_param,new_feat)
    bt(new_models,'train')
    bt(new_models,'validation')
    metric(new_models)
    refine(new_models,models)
    return new_models
  else:
    return []

def scores(model,features,name):
  y_pred = model.predict(X_test[features])
  precision = precision_score(y_test,y_pred)
  f1 = f1_score(y_test,y_pred)
  roc_auc = roc_auc_score(y_test,y_pred)
  mcc = matthews_corrcoef(y_test,y_pred)
  return pd.Series({'Precision':precision,'F1':f1,'ROC_AUC':roc_auc,'MCC':mcc},name=name)

time: 10.1 ms (started: 2023-03-06 17:24:43 +00:00)


##Data preprocessing

We will use a Kaggle datset to predict whether a reservation will be canceled. We use one-hote encoding to be able to pass categorical features to ```GradientBoostingClassifier```.  We need to drop some samples since their date is wrong.

In [3]:
df = pd.read_csv('/content/drive/MyDrive/kaggle/Hotel Reservations.csv/Hotel Reservations.csv')
df = df.rename({'arrival_year':'year','arrival_month':'month','arrival_date':'day'},axis=1)
df = df[(df['day']<29)|(df['month']!=2)]
df = df.set_index('Booking_ID')
df['booking_status'] = np.where(df['booking_status']=='Not_Canceled',1,0)

df = pd.get_dummies(df,drop_first=True)
df

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,year,month,day,repeated_guest,...,room_type_reserved_Room_Type 2,room_type_reserved_Room_Type 3,room_type_reserved_Room_Type 4,room_type_reserved_Room_Type 5,room_type_reserved_Room_Type 6,room_type_reserved_Room_Type 7,market_segment_type_Complementary,market_segment_type_Corporate,market_segment_type_Offline,market_segment_type_Online
Booking_ID,,,,,,,,,,,,,,,,,,,,,
INN00001,2,0,1,2,0,224,2017,10,2,0,...,0,0,0,0,0,0,0,0,1,0
INN00002,2,0,2,3,0,5,2018,11,6,0,...,0,0,0,0,0,0,0,0,0,1
INN00003,1,0,2,1,0,1,2018,2,28,0,...,0,0,0,0,0,0,0,0,0,1
INN00004,2,0,0,2,0,211,2018,5,20,0,...,0,0,0,0,0,0,0,0,0,1
INN00005,2,0,1,1,0,48,2018,4,11,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INN36271,3,0,2,6,0,85,2018,8,3,0,...,0,0,1,0,0,0,0,0,0,1
INN36272,2,0,1,3,0,228,2018,10,17,0,...,0,0,0,0,0,0,0,0,0,1
INN36273,2,0,2,6,0,148,2018,7,1,0,...,0,0,0,0,0,0,0,0,0,1


time: 276 ms (started: 2023-03-06 15:19:26 +00:00)


We divide our dataset: train, validation and test sets.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('booking_status',axis=1),df['booking_status'],random_state=0)
X_train1, X_val, y_train1, y_val = train_test_split(X_train,y_train,random_state=0)

time: 66.6 ms (started: 2023-03-06 15:19:26 +00:00)


##Fit 
We want to choose the best hyperparameters among 256 options. Instead of doing crosvalidation we will fit once our 256 models.

In [5]:
params = {'learning_rate': [.01,.04,0.1,0.6],
  'max_depth': [2,3,4,6],
  'min_samples_leaf': [50,100,200,300],
  'n_estimators': [10, 50 , 100,200]}

model = fit(params,df.drop('booking_status',axis=1).columns)

time: 14min 10s (started: 2023-03-06 15:19:26 +00:00)


##Model selection
We take the top 100 models according to a binomial hypothesis test. To be precise we want to test if the distribution of classes predicted by our models matches the expected distribution, the one of the validation targets. 

In [6]:
p_train = len(y_train1[y_train1==1])/len(y_train1)
p_val = len(y_val[y_val==1])/len(y_val)

bt(model,'validation')

model_vpv = model.copy()
model_vpv.sort(reverse=True,key=lambda x: x['bt_validation'].pvalue)

best_val = model_vpv[0:100]

time: 683 ms (started: 2023-03-06 15:33:37 +00:00)


We take the top 50 models according to a binomial hypothesis test. The same procedure as above, but this time our expected distribution corresponds to train targets.

In [7]:
bt(best_val,'train')
model_tpv = best_val.copy()
model_tpv.sort(reverse=True,key=lambda x: x['bt_train'].pvalue)

best_train = model_tpv[0:50]

time: 709 ms (started: 2023-03-06 15:33:38 +00:00)


We use the statistics obtained by the binomial tests to determine whether our models are overfit. The key concept is that the binomial test tell us how far is our predicted distribution from the real distribution. We drop those models above the .6 percentile according to our metric.

In [8]:
metric(best_train)
best_train.sort(key=lambda x: x['overfit'])
winners = best_train[0:20]

time: 2.4 ms (started: 2023-03-06 15:33:38 +00:00)


###Raw Selection

We take the model with the less overfitting. We use its parameters to train a new model on the train and validation sets. We evalute it with the test set.

In [36]:
best_simple_params = winners[0]['param']
simple_features = winners[0]['features']
simple = GradientBoostingClassifier(**best_simple_params,random_state=0)
simple.fit(X_train,y_train)

simple_scores = scores(simple,features=simple_features,name='Model_binomial_test')
simple_scores

Precision    0.901567
F1           0.919093
ROC_AUC      0.864365
MCC          0.745456
Name: Model_binomial_test, dtype: float64

time: 4.77 s (started: 2023-03-06 17:32:27 +00:00)


### Feature Selection
Before comparing the above model with the model Grid Searh finds, we will extract another model using feature selection.

We have chosen 20 out of 256 models. Now, we improve them with feature selection. In this case we only have 27 features, so we can drop one at a time, the one with the less importance.

In [10]:
history = [winners]
begin = winners
while len(begin)>=1:
  next = drop(begin)
  history.append(next)
  begin = next

time: 24min 45s (started: 2023-03-06 15:33:49 +00:00)


Thus, we have a family of models whose input varies in size. We choose the one tha minimizes overfit.

In [11]:
model_fam = []
for i in range(len(winners)):
  model_i = [x[i] for x in history[:-1]]
  model_i.sort(key=lambda y:y['overfit'])
  model_fam.append(model_i)

best_fam = [x[0] for x in model_fam]
best_fam.sort(key=lambda y:y['overfit'])

time: 4.9 ms (started: 2023-03-06 15:58:35 +00:00)


Now, we train our model on the train and validation sets. Then we calculate four scores.

In [37]:
overall_best_params = best_fam[0]['param']
overall_best_features = best_fam[0]['features']
overall_best = GradientBoostingClassifier(**overall_best_params,random_state=0)
overall_best.fit(X_train[overall_best_features],y_train)

ob_scores =scores(overall_best,features=overall_best_features,name='Model_binomial_test_feature_selection')
ob_scores

Precision    0.896351
F1           0.916533
ROC_AUC      0.858326
MCC          0.736104
Name: Model_binomial_test_feature_selection, dtype: float64

time: 10.4 s (started: 2023-03-06 17:32:32 +00:00)


###Grid Search
We will compare our two models with the model obtained by a grid search using cv.


In [13]:
params = {'learning_rate': [.01,.04,0.1,0.6],
  'max_depth': [2,3,4,6],
  'min_samples_leaf': [50,100,200,300],
  'n_estimators': [10, 50 , 100,200]}

gb = GradientBoostingClassifier(random_state=0)
gs = GridSearchCV(gb,
                  params,
                  scoring={'precision':make_scorer(metrics.precision_score),
                           'f1':make_scorer(metrics.f1_score),
                           'roc_auc':make_scorer(metrics.roc_auc_score),
                           'mcc':make_scorer(metrics.matthews_corrcoef)},
                  refit=False)
gs.fit(X_train,y_train)

GridSearchCV(estimator=GradientBoostingClassifier(random_state=0),
             param_grid={'learning_rate': [0.01, 0.04, 0.1, 0.6],
                         'max_depth': [2, 3, 4, 6],
                         'min_samples_leaf': [50, 100, 200, 300],
                         'n_estimators': [10, 50, 100, 200]},
             refit=False,
             scoring={'f1': make_scorer(f1_score),
                      'mcc': make_scorer(matthews_corrcoef),
                      'precision': make_scorer(precision_score),
                      'roc_auc': make_scorer(roc_auc_score)})

time: 1h 12min 35s (started: 2023-03-06 15:58:42 +00:00)


Since we are interested in four scores, we must check the best model for each score.

In [14]:
rank_precision = gs.cv_results_['rank_test_precision']
best_precision = np.where(rank_precision==1)[0]
gs.cv_results_['params'][best_precision[0]]

{'learning_rate': 0.6,
 'max_depth': 6,
 'min_samples_leaf': 50,
 'n_estimators': 200}

time: 5.51 ms (started: 2023-03-06 17:11:18 +00:00)


In [15]:
rank_f1 = gs.cv_results_['rank_test_f1']
best_f1 = np.where(rank_f1==1)[0]
gs.cv_results_['params'][best_f1[0]]

{'learning_rate': 0.6,
 'max_depth': 6,
 'min_samples_leaf': 50,
 'n_estimators': 200}

time: 5.42 ms (started: 2023-03-06 17:11:18 +00:00)


In [16]:
rank_roc_auc = gs.cv_results_['rank_test_roc_auc']
best_roc_auc = np.where(rank_roc_auc==1)[0]
gs.cv_results_['params'][best_roc_auc[0]]

{'learning_rate': 0.6,
 'max_depth': 6,
 'min_samples_leaf': 50,
 'n_estimators': 200}

time: 5.59 ms (started: 2023-03-06 17:11:18 +00:00)


In [17]:
rank_mcc = gs.cv_results_['rank_test_mcc']
best_mcc = np.where(rank_mcc==1)[0]
gs.cv_results_['params'][best_mcc[0]]

{'learning_rate': 0.6,
 'max_depth': 6,
 'min_samples_leaf': 50,
 'n_estimators': 200}

time: 12.5 ms (started: 2023-03-06 17:11:18 +00:00)


In this case, there is a model that maximizes the all four scores. We train it with the original features. We evaluate it on the test set.

In [35]:
gb = GradientBoostingClassifier(**gs.cv_results_['params'][best_mcc[0]], random_state=0)
gb.fit(X_train,y_train)

gs_scores = scores(gb,features=X_train.columns,name='Model_grid_search')
gs_scores

Precision    0.909631
F1           0.920864
ROC_AUC      0.871790
MCC          0.753918
Name: Model_grid_search, dtype: float64

time: 18.6 s (started: 2023-03-06 17:31:24 +00:00)


### Comparison

Compared with the raw model (no feature selection), the model obtained by Grid Searh performs less tha 1% better. In average it performs only .6% better.

In other words, the model obtained using Binomial Test not only saved us 80% of computational costs but it performs just as well as the Grid Search model.

In [38]:
gs_scores-simple_scores

Precision    0.013280
F1           0.004332
ROC_AUC      0.013464
MCC          0.017815
dtype: float64

time: 31.2 ms (started: 2023-03-06 17:32:43 +00:00)


On the other hand, Grid search only performs 1% better than the model we found using feature selection.

In [48]:
gs_scores-ob_scores

Precision    0.013280
F1           0.004332
ROC_AUC      0.013464
MCC          0.017815
dtype: float64

time: 10.5 ms (started: 2023-03-06 18:31:28 +00:00)


###Conclusions

Hypothesis testing can help decrease the computational expenses involved in identifying a machine learning model.